In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Dense, Concatenate, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.utils import plot_model,to_categorical
from sklearn.model_selection import train_test_split

In [2]:
print(os.getcwd())

C:\Users\emera


In [3]:
IMG_SHAPE  = (256, 256)
BATCH_SIZE = 32

In [4]:
train_img_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest')

val_img_generator = ImageDataGenerator(
    rescale = 1./255)

In [6]:
maindir = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2'
dataset = pd.read_csv(maindir + "\\annotations_handheld.csv", usecols=[0, 1, 2, 3, 4],
                          names=['image', 'x1', 'y1', 'x2', 'y2'])
img_files = os.listdir(maindir + "\\images_handheld")
dataset['flag'] = dataset['x1'] + dataset['y1'] + dataset['x2'] + dataset['y2'] == 0

image_data = []
labels = []
i = 0
for file in img_files:
    i += 1
    name = dataset.loc[dataset['image'] == file]
    label = name.iloc[0, 5]
    l = 1
    if label :
        l = 0
    path = maindir + "\\images_handheld\\" + file
    img = load_img(path)
    img = img.resize(IMG_SHAPE)
    x = img_to_array(img)
    image_data.append(x)
    labels.append(l)
    if i > 1525:
        break

appendir = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2'
dataset = pd.read_csv(appendir + "\\annotations_test.csv", usecols=[0, 1, 2, 3, 4],
                          names=['image', 'x1', 'y1', 'x2', 'y2'])
img_files = os.listdir(appendir + "\\testset")
dataset['flag'] = dataset['x1'] + dataset['y1'] + dataset['x2'] + dataset['y2'] == 0
i = 0
for file in img_files:
    i += 1
    name = dataset.loc[dataset['image'] == file]
    label = name.iloc[0, 5]
    l = 1
    if label :
        l = 0
    path = appendir + "\\testset\\" + file
    img = load_img(path)
    img = img.resize(IMG_SHAPE)
    x = img_to_array(img)
    image_data.append(x)
    labels.append(l)
    if i > 260:
        break        


In [7]:
import h5py as h5py
h5_train_data = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2\CNN\Output\train_data.h5'
h5_train_labels = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2\CNN\Output\train_labels.h5'

h5f_data = h5py.File(h5_train_data, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(image_data))

h5f_label = h5py.File(h5_train_labels, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(labels))

h5f_data.close()
h5f_label.close()

In [5]:
import h5py as h5py
h5_train_data = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2\CNN\Output\train_data.h5'
h5_train_labels = r'F:\emeraldfile\2021-2022S1\Project\project\images_handheld.tar_2\CNN\Output\train_labels.h5'

h5f_data = h5py.File(h5_train_data, 'r')
h5f_label = h5py.File(h5_train_labels, 'r')
global_features_string = h5f_data['dataset_1']
global_labels_string = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()

In [6]:
x_train, x_test, y_train, y_test = train_test_split(np.array(global_features), np.array(global_labels), test_size=0.3)
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(1250, 256, 256, 3) (1250, 2) (537, 256, 256, 3) (537, 2)


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense
from tensorflow.keras.layers import BatchNormalization

def build_model():
    model = Sequential()
    
    model.add(MaxPooling2D(pool_size=(7, 1), padding='valid'))
    
    model.add(Conv2D(96, (3,3), input_shape=(256,256,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(scale=False, center=False))

    model.add(Conv2D(64, (3,3)))
    model.add(BatchNormalization(scale=False, center=False))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 8), padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(48, (3,3)))
    model.add(BatchNormalization(scale=False, center=False))
    model.add(Activation('relu'))
    
    model.add(Conv2D(32, (3,3)))
    model.add(BatchNormalization(scale=False, center=False))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(3, 4), padding='same'))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    return model

In [12]:
model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
from tensorflow.keras.utils import Progbar
epochs=25
for e in range(epochs*4):
    print('Epoch', e)
    print('Training...')
    progbar = Progbar(x_train.shape[0])
    batches = 0

    for x_batch, y_batch in train_img_generator.flow(x_train,y_train, batch_size=BATCH_SIZE, shuffle=True):
        loss,train_acc = model.train_on_batch(x_batch, y_batch)
        batches += x_batch.shape[0]
        if batches > x_train.shape[0]:
            break
        progbar.add(x_batch.shape[0], values=[('train loss', loss),('train acc', train_acc)])

Epoch 0
Training...
1250/1250 [==============================] - 59s 48ms/step - train loss: 1.5528 - train acc: 0.5520
Epoch 1
Training...
1250/1250 [==============================] - 58s 46ms/step - train loss: 0.9299 - train acc: 0.5768
Epoch 2
Training...
1250/1250 [==============================] - 58s 47ms/step - train loss: 0.8555 - train acc: 0.5880
Epoch 3
Training...
1250/1250 [==============================] - 59s 47ms/step - train loss: 0.8795 - train acc: 0.5760
Epoch 4
Training...
1250/1250 [==============================] - 59s 47ms/step - train loss: 0.8150 - train acc: 0.6032
Epoch 5
Training...
1250/1250 [==============================] - 58s 47ms/step - train loss: 0.7308 - train acc: 0.6232
Epoch 6
Training...
1250/1250 [==============================] - 59s 47ms/step - train loss: 0.7186 - train acc: 0.6280
Epoch 7
Training...
1250/1250 [==============================] - 59s 47ms/step - train loss: 0.6958 - train acc: 0.6576
Epoch 8
Training...
1250/1250 [=========

1250/1250 [==============================] - 58s 46ms/step - train loss: 0.4191 - train acc: 0.8048
Epoch 68
Training...
1250/1250 [==============================] - 58s 46ms/step - train loss: 0.4496 - train acc: 0.7832
Epoch 69
Training...
1250/1250 [==============================] - 59s 47ms/step - train loss: 0.4005 - train acc: 0.8136
Epoch 70
Training...
1250/1250 [==============================] - 59s 47ms/step - train loss: 0.3859 - train acc: 0.8224
Epoch 71
Training...
1250/1250 [==============================] - 58s 47ms/step - train loss: 0.3803 - train acc: 0.8288
Epoch 72
Training...
1250/1250 [==============================] - 58s 47ms/step - train loss: 0.3937 - train acc: 0.8168
Epoch 73
Training...
1250/1250 [==============================] - 58s 47ms/step - train loss: 0.3780 - train acc: 0.8328
Epoch 74
Training...
1250/1250 [==============================] - 58s 47ms/step - train loss: 0.4452 - train acc: 0.7976
Epoch 75
Training...
1250/1250 [=====================

In [14]:
validation_data = val_img_generator.flow(x_test,y_test,
    batch_size = BATCH_SIZE,
    shuffle = True)
loss, acc = model.evaluate(validation_data)
print('Loss: ', loss)
print('Accuracy: ', acc)

17/17 [==============================] - 3s 184ms/step - loss: 0.4378 - accuracy: 0.7896
Loss:  0.43779808282852173
Accuracy:  0.7895717024803162
